In [1]:
from openai import OpenAI

ollama_client = OpenAI(
    api_key="ollama",
    base_url="http://127.0.0.1:11434/v1/"
)

In [31]:
# 设置提示词
SYSTEM_PROMPT = """
你是一个资深的小红书爆款文案专家，擅长结合最新潮流和产品卖点，创作引人入胜、高互动、高转化的笔记文案。

你的任务是根据用户提供的产品和需求，生成包含标题、正文、相关标签和表情符号的完整小红书笔记。

文案风格需活泼、真诚、富有感染力。当完成任务后，请以JSON格式直接输出最终文案，格式如下：
```json
{
  "title": "小红书标题",
  "body": "小红书正文",
  "hashtags": ["#标签1", "#标签2", "#标签3", "#标签4", "#标签5"],
  "emojis": ["✨", "🔥", "💖"]
}
```
在生成文案前，请务必先思考并收集足够的信息。
"""

In [43]:
# agent 核心代码
import json
import re

def generate_rednote(product_name: str, tone_style: str = "活泼甜美", max_iterations: int = 5) -> str:
    """
    使用 DeepSeek Agent 生成小红书爆款文案。
    """
    
    print(f"\n🚀 启动小红书文案生成助手，产品：{product_name}，风格：{tone_style}\n")
    
    # 存储对话历史，包括系统提示词和用户请求
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"请为产品「{product_name}」生成一篇小红书爆款文案。要求：语气{tone_style}，包含标题、正文、至少5个相关标签和5个表情符号。请以完整的JSON格式输出，并使用markdown语法 ```json```包裹起来。"}
    ]
    
    try:
        # 调用 DeepSeek API，传入对话历史和工具定义
        response = ollama_client.chat.completions.create(
            model="deepseek-r1:1.5b",
            messages=messages
        )

        response_message = response.choices[0].message
        
        print(f"[模型生成结果] {response_message.content}")
            
        # --- START: 添加 JSON 提取和解析逻辑 ---
        json_string_match = re.search(r"```json\s*(\{.*\})\s*```", response_message.content, re.DOTALL)
        
        if json_string_match:
            extracted_json_content = json_string_match.group(1)
            try:
                final_response = json.loads(extracted_json_content)
                print("任务完成，成功解析最终JSON文案。")
                return json.dumps(final_response, ensure_ascii=False, indent=2)
            except json.JSONDecodeError as e:
                print(f"提取到JSON块但解析失败: {e}")
                print(f"尝试解析的字符串:\n{extracted_json_content}")
                messages.append(response_message) # 解析失败，继续对话
        else:
            # 如果没有匹配到 ```json 块，尝试直接解析整个 content
            try:
                final_response = json.loads(response_message.content)
                print("任务完成，直接解析最终JSON文案。")
                return json.dumps(final_response, ensure_ascii=False, indent=2)
            except json.JSONDecodeError:
                print("生成了非JSON格式内容或非Markdown JSON块，可能还在思考或出错。")
                messages.append(response_message) # 非JSON格式，继续对话
        # --- END: 添加 JSON 提取和解析逻辑 ---
            
    except Exception as e:
        print(f"调用 DeepSeek API 时发生错误: {e}")
        
    return "未能成功生成文案。"

In [ ]:
# 测试案例 1: 黑松露抗老面霜
product_name_1 = "BOSE 蓝牙降噪耳机"
tone_style_1 = "调皮活泼"
result_1 = generate_rednote(product_name_1, tone_style_1)

print(result_1)


🚀 启动小红书文案生成助手，产品：BOSE 蓝牙降噪耳机，风格：调皮活泼

